# Week 7 Exercises

This week we'll be exploring computer vision & neural networks by training a convolutional neural network (CNN) on ADDI (Automated computer-based Diagnosis system for Dermascopic Images) dataset.

In [ ]:
import os, scipy.ndimage
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

## Exercise 1: Visualize the Data

Check out some of the images from the dataset by running the code below! Feel free to change the indeces to look through other images.

In [ ]:
base = 'resources/addi/train'
def get_example(label, idx):
    img_name = os.listdir('%s/%s' % (base, label))[idx]
    path = '%s/%s/%s' % (base, label, img_name)
    img = scipy.ndimage.imread(path)
    
    plt.figure()
    plt.imshow(img)
    plt.show()
    
get_example('normal', 0), get_example('abnormal', 0)

## Exercise 2: Train a Simple CNN

Now, we'll create a simple CNN model with just convolutional and fully-connected layers on the task. Much of the code is heavily influenced by [the Keras CNN tutorial](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), so definitely check that out!

### Load the data
Below, we have code that loads the data as an ImageDataGenerator, which will make it convenient for the Keras framework to deal with. Notice that the train generator has more input parameters--these are called data augmentations and allow us to generalize better when we have little training data. Read more about data augmentation here.

Run the code below to load the data.

In [ ]:
def load_data(batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory('resources/addi/train', 
                                                        target_size=(150, 150),
                                                        batch_size=batch_size,
                                                        class_mode='binary')

    val_generator = test_datagen.flow_from_directory('resources/addi/val',
                                                            target_size=(150, 150),
                                                            batch_size=batch_size,
                                                            class_mode='binary')
    
    test_generator = test_datagen.flow_from_directory('resources/addi/test',
                                                      target_size=(150, 150),
                                                      batch_size=batch_size,
                                                      class_mode='binary')
    
    return train_generator, val_generator, test_generator

train_generator, val_generator, test_generator = load_data(16)

### Create your Model

Write the code for your simple CNN below. Check out the [Keras documentation on CNNs](https://keras.io/layers/convolutional/) for reference!

In [ ]:
def get_simple_cnn():
    model = Sequential()
    
    ## YOUR MODEL CODE HERE

    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

simple_model = get_simple_cnn()
simple_model.summary()

### Train the Model!

Run the code below to train your simple model on the training data, evaluating on the validation data at every epoch. You should get around 60% on the validation set with this model.

In [ ]:
def train_model(model, train_generator, validation_generator, epochs):
    model.fit_generator(train_generator,
                        epochs=epochs,
                        validation_data=validation_generator,)
    
train_model(simple_model, train_generator, val_generator, 5)

## Exercise 3: Train an Advanced CNN

Now, we'll create a more advanced CNN model by adding max-pooling and spatial batch-normalization

### Create your Model

Architect your advanced model below. Again, be sure to reference the [Keras CNN docs](https://keras.io/layers/convolutional/), as well as the [Keras Normalization docs](https://keras.io/layers/normalization/) (for the Spatial Batch Norm layer).

In [ ]:
def get_advanced_cnn():
    model = Sequential()
    
    ## YOUR MODEL CODE HERE

    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

advanced_model = get_advanced_cnn()
advanced_model.summary()

### Train the Model!

Run the code below to train your simple model on the training data, evaluating on the validation data at every epoch. You should be able to get around 70% accuracy on validation with this model.

In [ ]:
train_model(advanced_model, train_generator, val_generator, 10)

### Evalutate the Model

Evaluate your model on the test set below! The first number is the loss, and the second is the accuracy. You should be able to get around 70% accuracy on the test set with your advanced model!

In [ ]:
advanced_model.evaluate_generator(test_generator)